<a href="https://colab.research.google.com/github/danohu/gists/blob/main/authors/fdabrandao/chatgpt/christmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMPL Christmas Model created by ChatGPT
[![christmas.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/chatgpt/christmas.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/authors/fdabrandao/chatgpt/christmas.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/chatgpt/christmas.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/authors/fdabrandao/chatgpt/christmas.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/authors/fdabrandao/chatgpt/christmas.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/authors/fdabrandao/chatgpt/christmas.ipynb)](https://colab.ampl.com)

Description: Christmas model generated by ChatGPT

Tags: christmas, chatgpt, amplpy, highlights

Notebook author: Filipe Brandão <<fdabrandao@gmail.com>>

Model author: [ChatGPT](https://chat.openai.com/)

In [ ]:
# Install dependencies
!pip install -q amplpy pandas

In [ ]:
# Google Colab & Kaggle integration
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["highs", "cbc", "gurobi"], # modules to install
    license_uuid="default", # license to use
    g=globals()) # instantiate AMPL object and register magics

# ChatGPT & AMPL

The AMPL dev team spent some time this week asking [ChatGPT](https://chat.openai.com/) to write AMPL models. Sometimes the responses were not perfect, which is to be expected, but we would work WITH ChatGPT on lines of code that were incorrect and it LEARNED from its errors, corrected the model and provided us with an updated version.

Social media posts with video: [LinkedIn Post](https://www.linkedin.com/feed/update/urn:li:activity:7011711985515978754) | [Twitter Post](https://twitter.com/AMPLopt/status/1605944316807053312) | [YouTube Video](https://www.youtube.com/watch?v=I3XnV_B6fW4)

# Christmas model by ChatGPT

**Problem description, mathematical model, AMPL model and data copied directly from [ChatGPT](https://chat.openai.com/)**:

| ![AMPL Model](https://github.com/ampl/amplcolab/raw/3009ffb009d195bb40eeb0cfa21ca1ffbdd8f84d/authors/fdabrandao/chatgpt/christmas_model_small.png) | ![MathJax description](https://github.com/ampl/amplcolab/raw/3009ffb009d195bb40eeb0cfa21ca1ffbdd8f84d/authors/fdabrandao/chatgpt/christmas_decription_small.png)|
|-|-|

In this optimization problem, we are trying to choose the best gift for each person, given a set of people `PEOPLE` and a set of gifts `GIFTS`, in order to maximize the total happiness. Each gift has a certain cost, and we have a budget that we cannot exceed. Each person can only receive one gift. The decision variables `x[p,g]` represent whether each person `p` receives each gift `g`, and must be binary (either 0 or 1).

The mathematical formulation of the problem is as follows:

$$
\begin{aligned}
\text{maximize} \quad &\sum_{p \in \text{PEOPLE}} \sum_{g \in \text{GIFTS}} \text{happiness}_{p,g} \cdot x_{p,g} \\
\text{subject to} \quad &\sum_{g \in \text{GIFTS}} \text{cost}_g \cdot x_{p,g} \le \text{budget} \quad \forall p \in \text{PEOPLE} \\
&\sum_{g \in \text{GIFTS}} x_{p,g} = 1 \quad \forall p \in \text{PEOPLE} \\
&x_{p,g} \in \{0,1\} \quad \forall p \in \text{PEOPLE}, g \in \text{GIFTS}
\end{aligned}
$$

In [ ]:
%%ampl_eval
reset; # clear previous definitions
# define the set of people
set PEOPLE;

# define the set of gifts
set GIFTS;

# define the cost of each gift
param cost{GIFTS};

# define the happiness level for each person receiving each gift
param happiness{PEOPLE, GIFTS};

# define the budget for gift-giving
param budget;

# define the decision variables:
# x[p,g] is 1 if person p receives gift g, 0 otherwise
var x{PEOPLE, GIFTS} binary;

# the objective is to maximize total happiness
maximize total_happiness: sum{p in PEOPLE, g in GIFTS} happiness[p,g] * x[p,g];

# subject to the budget constraint:
# the total cost of the gifts must be less than or equal to the budget
subject to budget_constraint: sum{p in PEOPLE, g in GIFTS} cost[g] * x[p,g] <= budget;

# each person can only receive one gift
subject to one_gift_per_person{p in PEOPLE}: sum{g in GIFTS} x[p,g] = 1;

# Load data directly from python data structures using [amplpy](https://amplpy.readthedocs.io/)

In [ ]:
import pandas as pd
happiness_df = pd.DataFrame({
        "Book": [3, 2, 5, 1, 4],
        "Toy": [5, 2, 4, 3, 1],
        "Chocolate": [1, 3, 4, 5, 2],
        "Wine": [2, 5, 3, 4, 1],
        "Flowers": [4, 3, 1, 2, 5]
    }, index=["Alice", "Bob", "Carol", "Dave", "Eve"]
)

In [ ]:
ampl.set["PEOPLE"] = ["Alice", "Bob", "Carol", "Dave", "Eve"]
ampl.set["GIFTS"] = ["Book", "Toy", "Chocolate", "Wine", "Flowers"]
ampl.param["cost"] = {"Book": 10, "Toy": 20, "Chocolate": 5, "Wine": 15, "Flowers":  7}
ampl.param["budget"] = 50
ampl.param["happiness"] = happiness_df.transpose().unstack()

# Solve with [HiGHS](https://ampl.com/products/solvers/open-source-solvers/) and retrieve solution

In [ ]:
ampl.option["solver"] = "highs"
ampl.solve()
solution = ampl.get_data("{p in PEOPLE, g in GIFTS: x[p, g] > 0} x[p, g]").to_dict()
print("To maximize the total happiness:")
for person, gift in solution:
    print(f"- give {gift} to {person}.")

HiGHS 1.4.0: HiGHS 1.4.0: optimal solution; objective 24
0 simplex iterations
1 branching nodes
To maximize the total happiness:
- give Flowers to Alice.
- give Wine to Bob.
- give Book to Carol.
- give Chocolate to Dave.
- give Flowers to Eve.


# Change the budget and solve with [Gurobi](https://ampl.com/products/solvers/solvers-we-sell/gurobi/)

In [ ]:
ampl.param["budget"] = 100
ampl.option["solver"] = "gurobi"
ampl.solve()
solution = ampl.get_data("{p in PEOPLE, g in GIFTS: x[p, g] > 0} x[p, g]").to_dict()
print("To maximize the total happiness:")
for person, gift in solution:
    print(f"- give {gift} to {person}.")

Gurobi 10.0.0: Gurobi 10.0.0: optimal solution; objective 25
0 simplex iterations
To maximize the total happiness:
- give Toy to Alice.
- give Wine to Bob.
- give Book to Carol.
- give Chocolate to Dave.
- give Flowers to Eve.


# Change the budget once again and solve with [CBC](https://ampl.com/products/solvers/open-source-solvers/)

In [ ]:
ampl.param["budget"] = 30
ampl.option["solver"] = "cbc"
ampl.solve()
solution = ampl.get_data("{p in PEOPLE, g in GIFTS: x[p, g] > 0} x[p, g]").to_dict()
print("To maximize the total happiness:")
for person, gift in solution:
    print(f"- give {gift} to {person}.")

cbc 2.10.7: cbc 2.10.7: optimal solution; objective 21
0 simplex iterations
To maximize the total happiness:
- give Flowers to Alice.
- give Chocolate to Bob.
- give Chocolate to Carol.
- give Chocolate to Dave.
- give Flowers to Eve.


# Some useful information:

- You can see more models at the AMPL Model Colaboratory: https://colab.ampl.com
- You can ask for help at the AMPL Discourse Forum: https://discuss.ampl.com
- You can get a free AMPL Community Edition license: https://ampl.com/ce/
